## 국민건강보험 건강검진 정보는?

* 건강검진정보란 국민건강보험의 직장가입자와 40세 이상의 피부양자, 세대주인 지역가입자와 40세 이상의 지역가입자의 일반건강검진 결과와 이들 일반건강검진 대상자 중에 만40세와 만66세에 도달한 이들이 받게 되는 생애전환기건강진단 수검이력이 있는 각 연도별 수진자 100만 명에 대한 기본정보(성, 연령대, 시도코드 등)와 검진내역(신장, 체중, 총콜레스테롤, 혈색소 등)으로 구성된 개방데이터 중 만개의 데이터를 임의로 추출했습니다.

## EDA
* EDA는 [부스트코스] 파이썬으로 시작하는 데이터 사이언스 > 4. 건강검진 데이터로 가설검정하기 를 참고해 주세요.
https://www.edwith.org/boostcourse-ds-510/lectures/28143

## 분류 모델로 음주여부 예측하기

* 건강검진 센터에서 음주여부에 응답을 하지 않는 사람이 있다고 가정합니다.
* 검진 데이터를 바탕으로 음주여부를 예측한다면 건강한 생활습관을 가이드하는데 도움이 될 것 입니다.
* 분류기 모델을 사용해서 건강검진 데이터를 바탕으로 음주여부를 예측해 봅니다.



## 필요한 라이브러리 로드
* 영상과 프로젝트 코드 작성에 사용한 버전입니다.
    * pandas : 1.0.1
    * numpy : 1.18.1
    * scikit-learn(sklearn) : 0.22.1

In [ ]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy
# 시각화를 위한 seaborn, matplotlib.pyplot 을 로드합니다. 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## 셀프리뷰를 위한 파일 로드

In [ ]:
# classification_checker.py 를 import 해서 사용합니다.
# classification_checker 명칭을 줄여서 checker 라고 사용합니다.
import check_util.classification_checker as checker

In [ ]:
# 셀프리뷰 파일이 정상로드 되었는지 확인합니다.
checker.check_load_self_review()

## 한글폰트 사용하기

In [ ]:
# 한글폰트를 설정해 주지 않으면 그래프 상에서 한글이 깨져보입니다.
# 한글이 출력될 수 있도록 폰트 설정을 해줍니다.
# 운영체제별 설정을 위해 로드 합니다.
import os

# 윈도우, 맥 외의 OS는 별도로 설정해 주세요.
if os.name == 'posix':
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")
# 마이너스 폰트 깨지는 문제에 대한 대처
plt.rc("axes", unicode_minus=False)

In [ ]:
# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
# 폰트의 주변이 흐릿하게 보이는 것을 방지합니다.
%config InlineBackend.figure_format = 'retina'

## 데이터 불러오기

In [ ]:
df = pd.read_csv("data/health.csv")
df.shape

## EDA 

In [ ]:
df["음주여부"].value_counts()

In [ ]:
sns.countplot(data=df, x="음주여부")

## 데이터 전처리
### 결측치 여부 보기

In [ ]:
df.isnull().sum()

### 결측치 채우기
* 결측치가 있다면 머신러닝 알고리즘이 학습과 예측을 할 수 없습니다.
* 결측치를 대체하는 방법으로는 여러가지가 있습니다. 평균, 중앙, 최빈값 등으로 채우기도 하며
* 그룹화된 값으로 대표값을 찾아 대체해 주기도 합니다.
* 결측치가 일부라면 제거하기도 합니다. 
* 또, 머신러닝을 통해 예측해서 대체하기도 합니다.

In [ ]:
# 요단백의 빈도수를 세어봅니다.
df["요단백"].value_counts()

### <font color="red">TODO 1</font> 결측치를 최빈값으로 채우기
* <font color="blue">fillna</font>를 사용해서 요단백의 결측치를 <font color="red">최빈값</font>으로 채웁니다.

In [ ]:
# 요단백의 최빈값을 구해서 mode 라는 변수에 할당합니다.
mode = df["요단백"].mode()[0]
mode

In [ ]:
# 결측치를 채워주세요.
df["요단백"] = _________________ # 밑줄을 제거하고 코드를 작성해 주세요!

In [ ]:
# 결측치가 제대로 채워졌는지 확인합니다.
df["요단백"].isnull().sum()

#### <font color="blue">TODO 1 Self Review</font>

In [ ]:
# 요단백 결측치가 잘 채워졌는지 다음의 코드를 실행해서 확인합니다.
# 아래의 코드를 실행해서 확인을 해주어야 제출파일을 작성할 수 있으니 꼭 확인해 주세요!
checker.check_null_up(df)

### <font color="red">TODO 2</font> 결측치 중앙값으로 채우기
* "LDL콜레스테롤"의 결측치를 <font color="blue">중앙값</font>으로 채웁니다.

In [ ]:
sns.distplot(df.loc[(df["LDL콜레스테롤"].notnull()) & (df["음주여부"] == 0), "LDL콜레스테롤"])
sns.distplot(df.loc[(df["LDL콜레스테롤"].notnull()) & (df["음주여부"] == 1), "LDL콜레스테롤"])

In [ ]:
# "LDL콜레스테롤" 의 결측치를 제거한 값에서 중앙값을 구합니다.
ldl_median = _______________ # 밑줄을 제거하고 중앙값을 채워주세요!
ldl_median

* 아래의 코드를 완성해서 "LDL콜레스테롤"의 결측치를 <font color="blue">중앙값</font>으로 채웁니다.

In [ ]:
df["LDL콜레스테롤"] = df["LDL콜레스테롤"].fillna(ldl_median)
df["LDL콜레스테롤"].isnull().sum()

#### <font color="blue">TODO 2 Self Review</font>

In [ ]:
# LDL콜레스테롤의 결측치가 잘 채워졌는지 다음의 코드를 실행해서 확인합니다.
# 아래의 코드를 실행해서 확인을 해주어야 제출파일을 작성할 수 있으니 꼭 확인해 주세요!
checker.check_ldl_median(df)

### <font color="red">TODO 3</font> 결측치 일괄 채우기
*  <font color="blue">replace</font>를 사용해서 전체 결측치를 <font color="red">1</font>로 채웁니다.

In [ ]:
df = _______________ # 밑줄을 제거하고 코드를 작성해 주세요!
df.isnull().sum()

#### <font color="blue">TODO 3 Self Review</font>

In [ ]:
# 모든 결측치가 잘 채워졌는지 다음의 코드를 실행해서 확인합니다.
# 아래의 코드를 실행해서 확인을 해주어야 제출파일을 작성할 수 있으니 꼭 확인해 주세요!
checker.check_null(df)

### 정규분포 형태로 변환
* "총콜레스테롤" 수치의 분포를 그려보면 왼쪽으로 치우쳐서 그려집니다.
* 정규분포를 이루도록 로그변환을 해줍니다.

In [ ]:
sns.distplot(df["총콜레스테롤"])

In [ ]:
df["총콜레스테롤_log"] = np.log(df["총콜레스테롤"] + 1)
sns.distplot(df["총콜레스테롤_log"])

* "감마지티피"수치의 분포를 그려보면 "총콜레스테롤" 처럼 왼쪽으로 치우쳐서 그려집니다.
* 또한 너무 뾰족한 형태를 이루고 있습니다.
* 정규분포를 이루도록 로그변환을 해줍니다.

In [ ]:
sns.distplot(df["감마지티피"])

In [ ]:
df["감마지티피_log"] = np.log(df["감마지티피"] + 1)
sns.distplot(df["감마지티피_log"])

## 학습, 예측 데이터셋 나누기

### <font color="red">TODO 4</font> 데이터셋 나누기
*  <font color="blue">슬라이싱</font>을 사용해서 학습:예측 데이터를 <font color="red">8:2</font>로 나눕니다.
* train 데이터는 8000개 test 데이터는 2000개의 행을 갖도록 나눕니다.

In [ ]:
# 8:2 의 비율로 구하기 위해 전체 데이터의 행에서 80% 위치에 해당되는 값을 구해서 split_count 라는 변수에 담습니다.
split_count = int(df.shape[0] * 0.8)
split_count

In [ ]:
# 파이썬의 슬라이싱 기능을 통해 데이터를 나눕니다.
# 위에서 만든 split_count 변수를 사용해서 슬라이싱으로 데이터를 나눠주세요
train = ______________
test =  ______________

#### <font color="blue">TODO 4 Self Review</font>

In [ ]:
# 셀을 실행해서 데이터셋이 잘 나뉘었는지 확인합니다.
# 아래의 코드를 실행해서 확인을 해주어야 제출파일을 작성할 수 있으니 꼭 확인해 주세요!
checker.check_split_dataset(train, test)

## 학습, 예측에 사용할 컬럼

In [ ]:
# feature_names 라는 변수에 학습과 예측에 사용할 컬럼명을 가져옵니다.
feature_names = df.columns.to_list()
feature_names

In [ ]:
# feature_names 라는 변수에 학습과 예측에 사용할 컬럼명을 따로 가져옵니다.
# 모든 컬럼을 다 사용하지 않습니다. 신호와 소음을 가려냅니다.
feature_names = [
 '성별코드',
 '연령대코드(5세단위)',
 '시도코드',
 '신장(5Cm단위)',
 '체중(5Kg 단위)',
 '허리둘레',
 '시력(좌)',
 '시력(우)',
 '수축기혈압',
 '이완기혈압',
 '식전혈당(공복혈당)',
 '총콜레스테롤_log',
 '트리글리세라이드',
 'HDL콜레스테롤',
 'LDL콜레스테롤',
 '혈색소',
 '요단백',
 '혈청크레아티닌',
 '(혈청지오티)AST',
 '(혈청지오티)ALT',
 '감마지티피_log',
 '흡연상태'
]

## 정답값이자 예측해야 될 값

In [ ]:
# label_name 이라는 변수에 예측할 컬럼의 이름을 담습니다.
label_name = "음주여부"
label_name

## 학습, 예측 데이터셋 만들기

In [ ]:
# 학습 세트 만들기 예) 기출문제
X_train = train[feature_names]
print(X_train.shape)
X_train.head()

In [ ]:
# 정답 값을 만들어 줍니다. 예) 기출문제의 정답
y_train = train[label_name]
print(y_train.shape)

In [ ]:
# 예측에 사용할 데이터세트를 만듭니다. 예) 실전 시험 문제
X_test = test[feature_names]

In [ ]:
# 예측의 정답값 예) 실전 시험 문제의 정답
y_test = test[label_name]
print(y_test.shape)

## 머신러닝 알고리즘 사용하기
* [랜덤 포레스트 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8)

### <font color="red">TODO 5</font> 랜덤포레스트 분류기를 만들어 주세요.
* model 이라는 변수에 정의해서 사용합니다.

In [ ]:
# 밑줄을 채워 코드를 완성해 주세요.
from sklearn._____ import ___________________

model = ___________________(n_estimators=1000,
                               min_samples_split=10, 
                               n_jobs=-1, 
                               random_state=42)
model

#### <font color="blue">TODO 5 Self Review</font>

In [ ]:
# 분류기가 잘 만들어졌는지 아래의 코드를 실행해서 확인해 주세요.
# 아래의 코드를 실행해서 확인을 해주어야 제출파일을 작성할 수 있으니 꼭 확인해 주세요!
checker.valid_model(model)

## 학습(훈련)
* 시험을 볼 때 기출문제(X_train)와 정답(y_train)을 보고 공부하는 과정과 유사합니다.

In [ ]:
model.fit(X_train, y_train)

## 예측
* 실전 시험문제(X_test)라고 보면 됩니다. 우리가 정답을 직접 예측합니다.

In [ ]:
y_predict = model.predict(X_test)
y_predict[:5]

## 트리 알고리즘 분석하기
* 피처의 중요도를 시각화 합니다.

In [ ]:
# 피처의 중요도를 추출하기
feature_importance = model.feature_importances_
feature_importance

In [ ]:
# 피처의 중요도 시각화 하기
pd.Series(feature_importance, index=feature_names).plot.barh(figsize=(8, 10))

## 정확도(Accuracy) 측정하기

In [ ]:
# 실제값 - 예측값을 빼주면 같은 값은 0으로 나오게 됩니다. 
# 여기에서 절대값을 씌운 값이 1인 값이 다르게 예측한 값이 됩니다.
# diff_count 에 값을 넣어줍니다.
diff_count = abs(test[label_name] - y_predict).sum()
diff_count

In [ ]:
# 예측의 정확도를 구합니다. 100점 만점 중에 몇 점을 맞았는지 구한다고 보면 됩니다.
((len(y_predict) - diff_count) / len(y_predict)) * 100

### <font color="red">TODO 6</font> 사이킷런의 Accuracy 로 정확도를 측정해 주세요.
* 사이킷런의 accuracy는 metrics 의 accuracy_score 를 사용해서 구할 수 있습니다.

In [ ]:
# 위에서 처럼 직접 구할 수도 있지만 미리 구현된 알고리즘을 가져와 사용합니다.
# 밑줄을 완성해서 accuracy를 측정해 주세요.
from sklearn.______ import ______________

score = ______________(y_test, y_predict) * 100
score

#### <font color="blue">TODO 6 Self Review</font>

In [ ]:
# 스코어를 체크해 주세요.
# 아래의 코드를 실행해서 확인을 해주어야 제출파일을 작성할 수 있으니 꼭 확인해 주세요!
checker.check_score(score)

## 프로젝트 제출 파일 만들기
* 아래의 코드를 실행하기 전에 <font color="red">실습한 주피터 노트북을 저장 버튼을 누르거나 다음의 단축키로 (ctrl/cmd + s) 저장</font>해 주세요. 
* 아래의 코드를 실행해 주세요. 
* 프로젝트 제출 파일이 생성됩니다.
* `제출파일 자동생성`은 주피터 익스텐션이 설치되어 있어야 실행이 가능합니다.
* 주피터 익스텐션은 다음의 방법으로 설치 가능합니다.
* 주피터 노트북의 `홈화면에서 > 오른쪽 상단 > New > Ternimal`로 설치해 주세요.
    * pip install jupyter_contrib_nbextensions
    * conda install jupyter_contrib_nbextensions
* submit 폴더에 다음 두 개의 파일이 생성되어 있어야 제출 파일이 제대로 생성된 것입니다.
    * health-regression-project.html
    * regression_submission.csv
* html 파일이 submit 폴더에 생성되지 않았다면 주피터 노트북의 `메뉴 > File > Download as > HTML(.html)` 을 통해 html 파일을 생성해 주시고 shubmit 폴더에 저장 후 압축해서 제출해 주세요.

In [ ]:
# 아래의 코드를 실행해서 프로젝트가 잘 작성되었는지 확인해 주세요.
# TODO 항목이 잘 구현되었다면 아래의 코드는 submit.zip 파일로 제출파일을 생성합니다.
# TODO 항목을 만족하지 않는다면 구현해야할 목록이 출력됩니다.
# 실행시간이 조금 오래 걸릴 수도 있습니다. 오류가 나지 않는다면 최대 30초 정도 기다려주세요!
checker.check_submit()